# Đồ án môn Programming for Data Science

### Sinh viên
    1.18120210            Phạm Tống Bình Minh
    2.18120209            Phạm Nhật Minh

## Import thư viện

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:

pd.set_option('max_colwidth', None)
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
#pd.set_option('display.width', None)

In [ ]:
plt.style.use('Solarize_Light2')

## Nguồn dữ liệu

Nhóm em sử dụng dữ liệu nói về thông tin khách hàng của một công tin viễn thông cung cấp dịch vụ điện thoại và internet cho 7-43 hộ dân tại Califonia trong quý 3.Thông tin cho biết khách hàng nào đã rời đi, ở lại hoặc đăng ký dịch vụ của công ty.

Dữ liệu này được public trên Kaggle([Telco Customer Churn](https://www.kaggle.com/blastchar/telco-customer-churn)). Phần đầy đủ của dữ liệu tại [IBM](https://community.ibm.com/community/user/businessanalytics/blogs/steven-macko/2019/07/11/telco-customer-churn-1113)

## Đọc dữ liệu

In [ ]:
df = pd.read_csv("data.csv")

In [ ]:
df.head()

## Khám phá dữ liệu

In [ ]:
df.shape

Dữ liệu có 7043 dòng và 21 cột

### Dữ liệu có các dòng bị lặp không?

In [ ]:
df.duplicated().sum()

dữ liệu không bị lặp

### Xem ý nghĩa của các cột

Dưới đây là phần mô tả của các cột trong tập dữ liệu

- **CustomerID**: Định danh duy nhất mỗi khách hàng
- **gender** : Giới tính (male or female)
- **SeniorCitizen**: Khách hàng là người cao tuổi (1: có , 0: không)
- **Partner**: Khách hàng có vợ hoặc chồng hay không (Yes, No)
- **Dependents**: Khách hàng có người phụ thuộc hay không (Yes, No)
- **tenure**: Số tháng khách hàng duy trì dịch vụ với công ty 
- **PhoneService**: Khách hàng có sử dụng dịch vụ điện thoại hay không
- **MultipleLines**: Khách hàng có sử dụng dịch vụ "MultipleLines" hay không (Yes, No, No phone service)
- **InternetService**: Khách hàng có sử dụng dịch vụ Internet hay không (DSL, Fiber optic, No)
- **OnlineSecurity**: Khách hàng có sử dụng dịch vụ "OnlineSecurity" hay không (Yes, No, No internet service)
- **OnlineBackup**: Khách hàng có sử dụng dịch vụ "OnlineBackup" hay không (Yes, No, No internet service)
- **DeviceProtection**: Khách hàng có sử dụng dịch vụ "DeviceProtection" hay không (Yes, No, No internet service)
- **TechSupport**: Khách hàng có sử dụng dịch vụ "TechSupport" hay không (Yes, No, No internet service)
- **StreamingTV**: Khách hàng có sử dụng dịch vụ "StreamingTV" hay không (Yes, No, No internet service)
- **StreamingMovies**: Khách hàng có sử dụng dịch vụ "StreamingMovies" hay không (Yes, No, No internet service)
- **Contract**: Thời hạn hợp động dịch vụ (Month-to-month, One year, Two year)
- **PaperlessBilling**: Hóa đơn điện tử (Yes or No)
- **PaymentMethod**: Phương thức thanh toán (Electronic check, Mailed check, Bank transfer (automatic), Credit card (automatic))
- **MonthlyCharges**: Chi phí dịch vụ hàng tháng
- **TotalCharges**: Tổng chi phí dịch phụ khách hàng trả
- **Churn**: Khách hàng có cắt hợp đồng dịch vụ không

### Xem tỉ lệ dữ liệu bị thiếu

In [ ]:
def missing_ratio(df):
    return (df.isna().mean() * 100).round(1)

In [ ]:
missing = df.agg(missing_ratio)
missing[missing > 0]

Có vẻ như không có cột nào bị thiếu

### Xem kiểu dữ liệu của các cột

In [ ]:
df.dtypes

+ Ta thấy cột TotalCharge là dạng object nên ta cần chuyển về kiểu float ở phần Tiền xử lý

## Tiền xử lý

#### chuyển cột "TotalCharges" từ object thành float64

In [ ]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

#### check lại tỉ lệ bị thiếu

In [ ]:
missing = df.agg(missing_ratio)
missing[missing > 0]

#### Nhận xét:

    + Vì dữ liệu bị thiếu ở cột TotalChagre không nhiều và dữ liệu còn lại khá lớn nên ta xóa các dòng bị khuyết

In [ ]:
df.dropna(inplace= True)

### Với mỗi cột có kiểu dữ liệu có dạng số, các giá trị được phân bố như thế nào?

In [ ]:
num_cols = df.select_dtypes(include = 'number').columns
num_cols

In [ ]:
def lower_quartile(df):
    return df.quantile(0.25).round(1)
def median(df):
    return df.quantile(0.5).round(1)
def upper_quartile(df):
    return df.quantile(0.75).round(1)

In [ ]:
df[num_cols].agg([min,lower_quartile, median, upper_quartile,max])

##### Nhận xét:
    + Các cột giá trị số không có gì khác thường

### Với mỗi cột có kiểu dữ liệu không phải dạng số, các giá trị được phân bố như thế nào?

In [ ]:
object_cols = df.select_dtypes(exclude = ['number']).columns

In [ ]:
def num_values(df):
    return df.nunique()
def value_ratios(c):
    return dict((c.value_counts(normalize=True) * 100)[0:5].round(3))
df[object_cols].agg([num_values, value_ratios])
df[object_cols].agg([num_values, value_ratios])

##### Nhận xét:
- Ta thấy ở cột Phone Service có tỉ lệ "No" là 9.67% và cột MultipleLines có tỉ lệ "No phone service" cũng là 9.67%. Điều này hiển nhiên vì nếu không đăng ký dịch vụ "PhoneService" thì không sử dụng được dịch vụ MutipleLines
- Ta thấy các cột "OnlineSecurity","OnlineBackup","DeviceProtection","TechSupport","StreamingTV","StreamingMovies" đều có tỉ lệ "No internet service" bắng 21.615% bằng với cột "InternetService" có tỉ lệ "No" là 21.615%. Điều này suy ra được là InternetServer là điều kiện cần để đăng ký các dịch vụ nêu trên.

## Phân tích các Insights

### Insight 1 : Lý do khiến người dùng thay đổi dịch vụ ( churn = "yes")

### Lấy data với churn = "yes" và churn = "no"

In [ ]:
churn_yes_df = df[df["Churn"] =='Yes']
churn_no_df = df[df["Churn"] =='No']

### Tính thời gian gắn bó (tenure)

In [ ]:
hist = plt.hist(churn_yes_df['tenure'],  
                edgecolor='black', bins = range(0,73,12), color=['orange'])
plt.xticks(range(0,73,12))
plt.title('Tenure histogram of churning users')
plt.xlabel('Tenure (months)')
plt.ylabel('Number of customers');

### Nhận xét:
    + Biểu đồ thể hiên với thời gian gắn bó ngắn (<12 tháng) thì sẽ tăng tỉ lệ churn
    + Ngược lại khi người dùng đã gắn bó lâu với dịch vụ thì tỉ lệ churn sẽ ít hơn.
 => Có thể nhận định rằng khoảng thời gian 12 tháng đầu là quan trọng nhất để giữ chân khách hàng với dịch vụ

### Phí hàng tháng (Monthly Charge)

In [ ]:
sns.kdeplot(churn_yes_df["MonthlyCharges"],color='red',label='Churn: Yes')
plt.legend()
plt.title("Monthly charges (with churn = yes) ")
plt.xlabel("Monthly charges")
plt.show()

### Nhận xét :
- Từ 70 đến 90 là mức phí hàng tháng quá cao khiến người dùng thay đổi dịch vụ.

### Tổng chi phí (Total Charge)

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(25,8))

sns.kdeplot(churn_yes_df["TotalCharges"],color='red',label='Churn: Yes' , ax=axs[0])
sns.kdeplot(churn_yes_df["tenure"],color='red',label='Churn: Yes' , ax=axs[1])

fig.suptitle('TotalCharge and Tenure (with churn= yes)', fontsize=18)
plt.show()

### Nhận xét :
- Với tổng chi phí < 2000 đô thì lượng người thay đổi dịch vụ chiếm tỉ trọng cao
- Tuy nhiên ta so sánh với biểu đồ thời lượng sử dụng thì thấy điểm tương đồng

=> Với thời lượng sử dụng dịch vụ ngắn dẫn đến tổng chi phí ngắn

### Dịch vụ liên quan Internet

In [ ]:
churn_yes_df["InternetService"].agg([ value_ratios])

Trong số những người thay đổi dịch vụ (churn = yes) có sử dụng dịch vụ internet chiếm đến 94% (tỉ lệ No chỉ 6%) nên ta xem các dịch vụ liên quan internet có gì bất thường không?

### Gói cước Internet + điện thoại

In [ ]:
services=[ ('PhoneService','Yes'),('MultipleLines','No'),('MultipleLines','No phone service'),('MultipleLines','Yes'), ('InternetService','DSL'),('InternetService','Fiber optic'), ('InternetService','No')]
services_churn_no_p=[len(churn_no_df[churn_no_df[k[0]] == k[1]]) for k in services] 
services_total_p=[len(df[df[k[0]] == k[1]]) for k in services]
services_labels = (k[0]+'_'+k[1] for k in services)

x = np.arange(len(services))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, services_churn_no_p, width, label='Non-churning users', color=['orange'])
rects2 = ax.bar(x + width/2, services_total_p, width, label='Total users', color=['#3fced9'])

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('User count')
ax.set_title('User count by product')
ax.set_xticks(x)
ax.set_xticklabels(services_labels)
ax.legend()

ax.bar_label(rects1, padding=3)
ax.bar_label(rects2, padding=3)

fig.set_figheight(10)
fig.set_figwidth(20)

plt.show()

### Nhận xét:
- Internet cáp quang churn nhiều hơn các dịch vụ khác
- Khách hàng không sử dụng internet churn rất ít

#### Tiền xử lý dữ liệu

In [ ]:
def Yes(c):
    return (c.value_counts(normalize=True)['Yes']* 100).round(3)
def No(c):
     return (c.value_counts(normalize=True)['No']* 100).round(3)
    
Internet_churn_yes_df = churn_yes_df[churn_yes_df["InternetService"] != "No"]
cols = ["OnlineSecurity","OnlineBackup","DeviceProtection","TechSupport","StreamingTV","StreamingMovies"]
Internet_churn_yes_df = Internet_churn_yes_df[cols].agg([Yes,No])
Internet_churn_yes_df = Internet_churn_yes_df.T
Internet_churn_yes_df

#### Biểu đồ thể hiện các dịch vụ liên quan đến internet

In [ ]:
Internet_churn_yes_df.plot.bar(stacked=True, figsize=(10,8), ylabel="Percent", color=['orange', '#3fced9']);

### Nhận xét
- Biểu đồ cho thấy số người sử dụng "OnlineSecurity" và "TechSupport" khá thấp nên người dùng đã bỏ lỡ 2 dịch vụ này dẫn đến không sử dụng dịch vụ Internet bên công ty nữa.
- Biểu đồ cho thấy số người sử dụng "StreamingTV" và "StreamingMovies" cao nhất trong các dịch vụ (>40%) ta nên hỏi ý kiến khách hàng xem có gì không hài lòng dẫn đến việc rời bỏ dịch vụ Internet bên công ty 

### Kết luận :
#### Các lý do khiến người dùng thay đổi dịch vụ:
- Do thời gian gắn bó chưa lâu (<10 tháng) vì vậy cần lắng nghe và chăm sóc khách hàng mới
- Do chi phí hàng tháng quá cao từ ( 70 -> 90) dẫn đến các khách hàng chọn rời dịch vụ
- Do chất lượng dịch vụ Internet và các tính năng của dịch vụ chưa được tốt dẫn đến các khách hàng chọn rời dịch vụ

### Insight 2: Customer profile và các góp ý giúp chăm sóc khách hàng tốt hơn

#### Lọc ra các khách hàng cao tuổi

In [ ]:
senior_df = df[df['SeniorCitizen'] == 1]
object_cols = senior_df.select_dtypes(exclude = ['number']).columns
def num_values(df):
    return df.nunique()
def value_ratios(c):
    return dict((c.value_counts(normalize=True) * 100)[0:5].round(3))
senior_df[object_cols].agg([num_values, value_ratios])

#### Profile khách hàng cao tuổi:
- Có nhu cầu sử dụng internet cao (95% có internet)
- Thường đăng kí hợp đồng từng tháng (70%)
- Sử dụng hóa đơn điện tử nhiều (75% paperless billing)

#### Lọc ra các khách hàng cao tuổi đã churn

In [ ]:
senior_churn_df = senior_df[senior_df["Churn"] == "Yes"]
senior_churn_df[object_cols].agg([num_values, value_ratios])

#### Phân tích churn của khách hàng cao tuổi:
- Phần lớn đăng kí hợp đồng tháng (92%)
- Sử dụng internet cáp quang nhiều (82%)
- Ít sử dụng thanh toán tự động hơn

#### Gợi ý về chăm sóc khách hàng:
- Đưa ra ưu đãi về giá cho hợp đồng dài hạn -> khuyến khích đăng kí dài hạn, giảm gánh nặng chi phí (phân khúc người già churn nhiều do tiết kiệm)
- Đưa ra các gói dịch vụ thiết yếu ưu đãi cho người lớn tuổi (vd Internet + Điện thoại + Tech support) để thu hút khách hàng
- Khuyến mãi hoặc giảm giá các dịch vụ có ích cho người lớn tuổi như Tech support và Online security để tăng giá trị cho khách hàng
- Cải thiện chất lượng dịch vụ internet

#### Lọc ra profile khách hàng trẻ (non-senior)

In [ ]:
non_senior_df = df[df['SeniorCitizen'] == 0]
object_cols = non_senior_df.select_dtypes(exclude = ['number']).columns
def num_values(df):
    return df.nunique()
def value_ratios(c):
    return dict((c.value_counts(normalize=True) * 100)[0:5].round(3))
non_senior_df[object_cols].agg([num_values, value_ratios])

#### Profile khách hàng trẻ tuổi:
- Một bộ phận không đăng kí internet (25% sử dụng 4G hoàn toàn?)
- Sử dụng các dịch vụ kèm theo nhiều hơn (đặc biệt là streaming)
- Đăng kí hợp đồng dài hạn nhiều hơn

#### Lọc ra các khách hàng trẻ tuổi đã churn

In [ ]:
non_senior_churn_df = non_senior_df[non_senior_df["Churn"] == "Yes"]
non_senior_churn_df[object_cols].agg([num_values, value_ratios])

#### Phân tích churn của khách hàng trẻ tuổi:
- Phần lớn đăng kí hợp đồng tháng
- Sử dụng internet cáp quang nhiều
- Ít sử dụng các dịch vụ đi kèm như tech support

#### Gợi ý về chăm sóc khách hàng:
- Đưa ra ưu đãi về giá cho hợp đồng dài hạn -> khuyến khích đăng kí dài hạn
- Ưu đãi lắp đặt internet để tăng phủ sóng internet cho khách hàng
- Đưa ra các gói dịch vụ ưu đãi cho người trẻ (VD Internet + Streaming TV + Streaming Movies) để thu hút khách hàng
- Cải thiện chất lượng dịch vụ internet

### Kết luận :
#### Gợi ý về chăm sóc khách hàng nói chung:
- Đưa ra ưu đãi về giá cho hợp đồng dài hạn -> khuyến khích đăng kí dài hạn
- Cải thiện chất lượng dịch vụ streaming nói riêng và internet nói chung
- Thường xuyên lắng nghe phản hồi cũng như hỗ trợ khách hàng về các dịch vụ

#### Gợi ý về chăm sóc khách hàng lớn tuổi:
- Đưa ra các gói dịch vụ thiết yếu ưu đãi cho người lớn tuổi (vd Internet + Điện thoại + Tech support) để thu hút khách hàng
- Khuyến mãi hoặc giảm giá các dịch vụ có ích cho người lớn tuổi như Tech support và Online security để tăng giá trị cho khách hàng

#### Gợi ý về chăm sóc khách hàng trẻ tuổi:
- Ưu đãi lắp đặt internet để tăng phủ sóng internet cho khách hàng
- Đưa ra các gói dịch vụ ưu đãi cho người trẻ (VD Internet + Streaming TV + Streaming Movies) để thu hút khách hàng

## Nhìn lại quá trình làm đồ án 

- Cần tìm hiểu kĩ nguồn gốc data hơn
- Nên áp dụng một thuật toán vào phân tích dữ liệu để có thể đưa ra thêm giải pháp

## Tài liệu tham khảo
- Kaggle([Telco Customer Churn](https://www.kaggle.com/blastchar/telco-customer-churn))
- https://matplotlib.org/
- https://pandas.pydata.org/